<a href="https://colab.research.google.com/github/p4arth/Recommendation-Systems-Learning/blob/main/ZomatoReccomendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd 
import numpy as np

In [15]:
rest = pd.read_csv('/content/drive/MyDrive/DelhiNCR Restaurants.csv')

In [16]:
rest.head()

,Restaurant_Name,Category,Pricing_for_2,Locality,Dining_Rating,Dining_Review_Count,Delivery_Rating,Delivery_Rating_Count,Website,Address,Phone_No,Latitude,Longitude,Known_For2,Known_For22
0,Rustom's,"Parsi, Street Food, North Indian, Desserts, Be...",2100,"ITO, New Delhi",4.9,1885,4.4,1844,https://www.zomato.com/ncr/rustoms-ito-new-delhi,"Delhi Parsi Anjuman, LNJP Colony, Bahadur Shah...",9.1991E+11,28.638167,77.240472,NaN,NaN
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200,"Pragati Maidan, New Delhi",4.9,3748,3.9,37,https://www.zomato.com/ncr/cafe-lota-pragati-m...,"National Crafts Museum, Gate 2, Bhairon Marg, ...",9.17839E+11,28.613429,77.242471,"Pondicherry Fish Curry, Coconut Rabdi, Sabudan...","Artistic Decor, The Service, Natural Ambience,..."
2,Dum-Pukht - ITC Maurya,"Mughlai, North Indian, Desserts",5000,"ITC Maurya, Chanakyapuri, New Delhi",4.9,1371,NaN,0,https://www.zomato.com/ncr/dum-pukht-itc-maury...,"ITC Maurya, Chanakyapuri, New Delhi",9.11146E+11,28.598200,77.173661,"Royal ambiance and Mughlai cuisine, also recei...",NaN
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,2636,4.4,1238,https://www.zomato.com/ncr/burma-burma-dlf-cyb...,"Shop 6, Ground Floor, Building 8, Tower C, Cyb...",9.11244E+11,28.494464,77.088532,"Coconut Milk Dessert, Honey Caviar, Kaffir Lim...","Knowledgeable Staff, Authentic, Soothing Ambie..."
4,The Big Chill,"Continental, American, Italian",1500,"Khan Market, New Delhi",4.9,6487,NaN,0,https://www.zomato.com/ncr/the-big-chill-khan-...,"68-A, Khan Market, New Delhi",9.11142E+11,28.600350,77.227493,"Irish Cream Tiramisu, Penne Vodka, Mississippi...","Retro Ambience, Yummy Desserts, Big Portions, ..."


In [17]:
rest.drop(['Pricing_for_2','Locality', 'Website', 'Address', 'Phone_No', 'Latitude', 'Longitude', 'Known_For22'], axis =1, inplace=True)

In [18]:
rest.head()

,Restaurant_Name,Category,Dining_Rating,Dining_Review_Count,Delivery_Rating,Delivery_Rating_Count,Known_For2
0,Rustom's,"Parsi, Street Food, North Indian, Desserts, Be...",4.9,1885,4.4,1844,NaN
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",4.9,3748,3.9,37,"Pondicherry Fish Curry, Coconut Rabdi, Sabudan..."
2,Dum-Pukht - ITC Maurya,"Mughlai, North Indian, Desserts",4.9,1371,NaN,0,"Royal ambiance and Mughlai cuisine, also recei..."
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",4.9,2636,4.4,1238,"Coconut Milk Dessert, Honey Caviar, Kaffir Lim..."
4,The Big Chill,"Continental, American, Italian",4.9,6487,NaN,0,"Irish Cream Tiramisu, Penne Vodka, Mississippi..."


In [19]:
rest.shape

(1965, 7)

In [20]:
rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1965 entries, 0 to 1964
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Restaurant_Name        1965 non-null   object 
 1   Category               1965 non-null   object 
 2   Dining_Rating          1965 non-null   float64
 3   Dining_Review_Count    1965 non-null   int64  
 4   Delivery_Rating        1563 non-null   float64
 5   Delivery_Rating_Count  1965 non-null   int64  
 6   Known_For2             1560 non-null   object 
dtypes: float64(2), int64(2), object(3)
memory usage: 107.6+ KB


In [21]:
rest['Delivery_Rating'] = rest['Delivery_Rating'].fillna(0)
rest['Known_For2'] = rest['Known_For2'].fillna('')

In [22]:
rest.head()

,Restaurant_Name,Category,Dining_Rating,Dining_Review_Count,Delivery_Rating,Delivery_Rating_Count,Known_For2
0,Rustom's,"Parsi, Street Food, North Indian, Desserts, Be...",4.9,1885,4.4,1844,
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",4.9,3748,3.9,37,"Pondicherry Fish Curry, Coconut Rabdi, Sabudan..."
2,Dum-Pukht - ITC Maurya,"Mughlai, North Indian, Desserts",4.9,1371,0.0,0,"Royal ambiance and Mughlai cuisine, also recei..."
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",4.9,2636,4.4,1238,"Coconut Milk Dessert, Honey Caviar, Kaffir Lim..."
4,The Big Chill,"Continental, American, Italian",4.9,6487,0.0,0,"Irish Cream Tiramisu, Penne Vodka, Mississippi..."


In [24]:
rest.head(2)['Known_For2']

0                                                     
1    Pondicherry Fish Curry, Coconut Rabdi, Sabudan...
Name: Known_For2, dtype: object

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 4, max_features=None,
                      strip_accents='unicode',analyzer='word',token_pattern='(?u)\\b\\w+\\b',
                      ngram_range=(1,3),
                      stop_words = 'english')

In [37]:
tfv_matrix = tfv.fit_transform(rest['Known_For2'])

In [38]:
tfv_matrix

<1965x1581 sparse matrix of type '<class 'numpy.float64'>'
	with 21977 stored elements in Compressed Sparse Row format>

In [47]:
tfv_matrix.shape

(1965, 1581)

In [39]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [41]:
indices = pd.Series(rest.index, index=rest['Restaurant_Name']).drop_duplicates()

In [42]:
indices

Restaurant_Name
Rustom's                     0
Cafe Lota                    1
Dum-Pukht - ITC Maurya       2
Burma Burma                  3
The Big Chill                4
                          ... 
Bablu Chic-Inn            1960
Kovilakam                 1961
Krips Restaurant          1962
Cafe 147                  1963
Pizzasia                  1964
Length: 1965, dtype: int64

In [48]:
def give_rec(restaurant, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[restaurant]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:11]

    # Movie indices
    rest_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return rest['Restaurant_Name'].iloc[rest_indices]

In [50]:
giv_rec("Glued Reloaded")

1963                              Cafe 147
1962                      Krips Restaurant
1961                             Kovilakam
1960                        Bablu Chic-Inn
1959                             Cocoberry
1958                        Glued Reloaded
1957       Ricks Bar - The Taj Mahal Hotel
1956                           Playboy Tap
1955                         Chilli Dragon
1954    Grills & Platters By Pind Balluchi
Name: Restaurant_Name, dtype: object